In [ ]:
pip install xlrd
pip install openpyxl
pip install pandas

In [2]:
import pymysql
conn = pymysql.connect(host='localhost', user='db_test', password='hyesu0429@', db='dbProject')

In [3]:
curs = conn.cursor(pymysql.cursors.DictCursor)

In [4]:
import pandas as pd
import numpy as np

fileName='/Users/mac/Downloads/movieInfo.xlsx'
df = pd.read_excel(fileName,skiprows = 4,nrows=10)
df = df.replace({np.nan : None})

display(df)

,영화명,영화명(영문),제작연도,제작국가,유형,장르,제작상태,감독,제작사
0,플래시,The Flash,2023,미국,장편,액션,개봉예정,안드레스 무시에티,None
1,엔니오: 더 마에스트로,Ennio,2021,이탈리아,장편,다큐멘터리,개봉예정,쥬세페 토르나토레,None
2,뉴 노멀,NEW NORMAL,2023,한국,장편,"스릴러,코미디",기타,정범식,None
3,명탐정코난: 하이바라 아이 이야기 ~흑철의 미스터리 트레인,Detective Conan: The Story of Ai Haibara: Blac...,2023,일본,장편,애니메이션,개봉예정,None,None
4,아기공룡 둘리-얼음별 대모험,The Adventure Of Ice Planet,1996,한국,장편,"애니메이션,어드벤처,코미디,가족",개봉,"김수정,임경원",(주)둘리나라
5,투란도트,Turandot,2022,독일,장편,공연,기타,필립 슈톨츨,None
6,토스카,Tosca,2022,독일,장편,공연,기타,마틴 쿠세이,None
7,나비부인,Madama Butterfly,2022,독일,장편,공연,기타,마이클 레빈,None
8,260: 부코바르 대학살,Sixth Bus,2022,기타,온라인전용,"드라마,전쟁",기타,None,None
9,군산전기,The City of Outlanders,2020,한국,장편,다큐멘터리,개봉예정,"문승욱,유예진",None


## 테이블 생성

In [5]:
sql_create_table_movie = "CREATE TABLE IF NOT EXISTS Movie ( id INT primary key NOT NULL AUTO_INCREMENT, name VARCHAR(45) NOT NULL, name_eng TEXT, production_year INT, type VARCHAR(10), production_status VARCHAR(10), production_company VARCHAR(100) )"
sql_create_table_director = "CREATE TABLE IF NOT EXISTS Director ( id INT primary key NOT NULL AUTO_INCREMENT, name VARCHAR(45) NOT NULL ); "
sql_create_table_movieDirector = "CREATE TABLE IF NOT EXISTS Movie_Director ( movie_id INT NOT NULL, director_id INT NOT NULL, PRIMARY KEY (movie_id, director_id), foreign key (movie_id) references Movie(id)  on update cascade on delete cascade, foreign key (director_id) references Director(id) on update cascade on delete cascade );"
sql_create_table_country = "CREATE TABLE IF NOT EXISTS Country ( id INT primary key NOT NULL AUTO_INCREMENT, name VARCHAR(20) NOT NULL );"
sql_create_table_movieCountry = "CREATE TABLE IF NOT EXISTS Movie_Country ( movie_id INT NOT NULL, country_id INT NOT NULL, PRIMARY KEY (movie_id, country_id), foreign key (movie_id) references Movie(id)  on update cascade on delete cascade, foreign key (country_id) references Country(id) on update cascade on delete cascade );"
sql_create_table_genre = "CREATE TABLE IF NOT EXISTS Genre ( movie_id INT NOT NULL, genre_id INT NOT NULL, name VARCHAR(20) NOT NULL, PRIMARY KEY (movie_id, genre_id), foreign key (movie_id) references Movie(id) on update cascade on delete cascade );"

curs.execute(sql_create_table_movie)
curs.execute(sql_create_table_director)
curs.execute(sql_create_table_movieDirector)
curs.execute(sql_create_table_country)
curs.execute(sql_create_table_movieCountry)
curs.execute(sql_create_table_genre)

0

## 엑셀 데이터 DB에 삽입

In [9]:
sql_insert_movie = "INSERT INTO Movie (id, name, name_eng, production_year, type, production_status, production_company) values(%s, %s, %s, %s, %s, %s, %s)"
sql_insert_director = "INSERT INTO Director (name) values(%s)"
sql_insert_movie_director= "INSERT INTO Movie_Director (movie_id, director_id) values(%s, %s)"
sql_insert_country = "INSERT INTO Country (name) values(%s)"
sql_insert_movie_country= "INSERT INTO Movie_Country (movie_id, country_id) values(%s, %s)"
sql_insert_genre= "INSERT INTO Genre (movie_id, name) values(%s, %s)"

tuples_movie = []
tuples_director = []
tuples_movie_director = []
tuples_country = []
tuples_movie_country = []
tuples_genre = []

directorList=[]
countryList=[]

for idx, row in df.iterrows():
    name = row['영화명']
    name_eng = row['영화명(영문)']
    production_year = row['제작연도']
    countries = row['제작국가']
    type = row['유형']
    genres = row['장르']
    production_status = row['제작상태']
    directors = row['감독']
    production_company = row['제작사']
    
    print("movie id: ", idx)

    tuples_movie.append((idx, name, name_eng, production_year, type, production_status, production_company))
    
    if directors:
        directors = directors.split(',')
        for director in directors:
            if director not in directorList:
                print("director id: ",len(directorList))
                tuples_director.append((len(directorList),director))
                tuples_movie_director.append((idx,len(directorList)))
                directorList.append(director)
            else:
                print("director id: ",directorList.index(director) )
                tuples_movie_director.append((idx,directorList.index(director)))
                
    if countries:
        countries = countries.split(',')
        for country in countries:
            if country not in countryList:
                tuples_country.append((len(countryList),country))
                tuples_movie_country.append((idx,len(countryList)))
                countryList.append(country)
            else:
                tuples_movie_country.append((idx,countryList.index(country)))
                
    if genres:
        genres = genres.split(',')
        for i,genre in enumerate(genres):
            tuples_genre.append((idx, i, genre))
            
            
curs.executemany(sql_insert_movie, tuples_movie)
curs.executemany(sql_insert_director, tuples_director)
curs.executemany(sql_insert_movie_director, tuples_movie_director)
curs.executemany(sql_insert_country, tuples_country)
curs.executemany(sql_insert_movie_country, tuples_movie_country)
curs.executemany(sql_insert_genre, tuples_genre)
conn.commit()

print("insert success")

movie id:  0
director id:  0
movie id:  1
director id:  1
movie id:  2
director id:  2
movie id:  3
movie id:  4
director id:  3
director id:  4
movie id:  5
director id:  5
movie id:  6
director id:  6
movie id:  7
director id:  7
movie id:  8
movie id:  9
director id:  8
director id:  9


IntegrityError: (1062, "Duplicate entry '1' for key 'movie.PRIMARY'")

In [7]:
# # director 테이블
# sql_insert_director = "INSERT INTO Director (name) values(%s)"

# tuples_director = []
# for idx, row in df.iterrows():
#     names = row['감독']
    
#     if names:
#         nameList = names.split(',')
#         for name in nameList:
#             if name not in directorNameSet:
#                 directorNameSet.add(name)
#                 tuples_director.append(name)
                         
# curs.executemany(sql_insert_director, tuples_director)
# conn.commit()

# tuples_director=[]
# print("director insert success")

In [ ]:
# movie-director 테이블

sql_select_movie = "select id,name from Movie"
curs.execute(sql_select_movie)

row = curs.fetchone()
while row:
    print(row)
    row = curs.fetchone()

#     curs.execute(sql_insert_director, a)
#     conn.commit()